# Filtrado en el dominio de la frecuencia - Filtrado Pasa-bajas

<div class="alert alert-block alert-success">
<b>Resumen:</b> Este notebook implementa un filtro pasabajas en el dominio de la frecuencia utilizando la Transformada de Fourier. El filtro permite preservar las bajas frecuencias de una imagen, eliminando las altas frecuencias que suelen corresponder a bordes y contornos.
</div>

***

Importación de librerias necesarias

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

## 2. Definición de Funciones Auxiliares
### 2.1. Creación de una máscara Gaussiana
Esta función crea una máscara Gaussiana que se utilizará para filtrar las frecuencias en el espectro de Fourier

In [ ]:
def create_lpf_mask(dft_shift, sigma):
    """
    Genera una máscara de filtro pasa bajas gaussiana con regularización.

    Parámetros:
    -----------
    dft_shift : ndarray
        Transformada de Fourier desplazada (2D).
    
    sigma : float
        Desviación estándar del filtro gaussiano.

    Retorna:
    --------
    H : ndarray
        Máscara de filtro pasa bajas de las mismas dimensiones que dft_shift.
    """
    
    # tamaño de la mascara del filtro
    rows, cols = dft_shift.shape[:2]

    # Crear el filtro gaussiano
    x = np.linspace(-cols//2, cols//2, cols)
    y = np.linspace(-rows//2, rows//2, rows)
    X, Y = np.meshgrid(x, y)
    D = np.sqrt(X**2 + Y**2)

    # Filtro pasa bajas usando el filtro gaussiano
    mask = np.exp(-(D**2 / (2 * (sigma ** 2))))

    return mask.astype(np.float32)

### 2.2. Reconstrucción de la imagen filtrada
Esta función reconstruye la imagen filtrada a partir del espectro de Fourier filtrado.

In [ ]:
def reconstruct_image(dft_shift_filtered):
    """
    Reconstruye la imagen filtrada a partir del espectro de Fourier filtrado.

    Args:
        dft_shift_filtered (np.ndarray): Espectro de Fourier filtrado.

    Returns:
        np.ndarray: Imagen filtrada en escala de grises.
    """
    # Desplazar el espectro de vuelta
    dft_inverse_shift = np.fft.ifftshift(dft_shift_filtered)
    
    # Aplicar la transformada inversa de Fourier
    img_back = cv2.idft(dft_inverse_shift)
  
    # Calcula la magnitud
    img_back = cv2.magnitude(img_back[:,:,0],img_back[:,:,1])
     
    # Normalizar para visualización
    img_back = cv2.normalize(img_back, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    
    return img_back

## 3. Carga y verificación de la Imagen

Cargamos la imagen en escala de grises y verificamos que sea correcta.

In [ ]:
# Se utiliza cv2.IMREAD_GRAYSCALE para simplificar el análisis de frecuencia
im_1 = cv2.imread('./images/lena_gray.tif', cv2.IMREAD_GRAYSCALE)

# Verificación de que la imagen se haya cargado correctamente
if im_1 is None:
    raise FileNotFoundError("La imagen no se encontró en la ruta especificada.")

## 4. Transformada de Fourier

Convertimos la imagen al dominio de la frecuencia utilizando la Transformada Discreta de Fourier (DFT) y centramos el espectro.

In [ ]:
# Convertir la imagen a float y usando DFT
dft = cv2.dft(np.float32(im_1), flags=cv2.DFT_COMPLEX_OUTPUT)

# Desplazar el espectro de Fourier para centrar las bajas frecuencias
dft_shift = np.fft.fftshift(dft)
print(f"Transformada de Fourier aplicada y desplazada (size={dft_shift.shape}).")

## 5. Aplicación del Filtro Pasabajas

Creamos y aplicamos la máscara circular para filtrar las altas frecuencias.

In [ ]:
# Crear máscara circular basada en el tamaño de dft_shift
sigma = 60
mask = create_lpf_mask(dft_shift, sigma)
print("Máscara creada basada en dft_shift.")

# Aplicar filtro pasabajas
# Expandir las dimensiones de la máscara para que coincida con el espectro de Fourier
dft_shift_filtered = dft_shift * mask[:,:,np.newaxis]
print("Filtro pasabajas aplicado al espectro de Fourier.")

## 6. Reconstrucción de la imagen filtrada

Utilizamos la función definida anteriormente para reconstruir la imagen filtrada a partir del espectro de Fourier filtrado.

In [ ]:
# Reconstrucción la imagen filtrada
im_filtered = reconstruct_image(dft_shift_filtered)

## 7. Visualización de la imagen original y filtrada

Comparamos la imagen original con la imagen filtrada para observar el efecto del filtro pasabajas.

In [ ]:
# Visualizar la imagen filtrada
plt.figure(figsize=(8,8))
plt.subplot(121),plt.imshow(im_1, cmap = 'gray')
plt.title('Imagen original'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(im_filtered, cmap = 'gray')
plt.title('Imagen filtrada pasabajas'), plt.xticks([]), plt.yticks([])
plt.show()

### 7.1 Visualización de la máscara

In [ ]:
plt.figure(figsize=(2,2))
plt.imshow(mask, cmap='gray')
plt.title('Máscara')
plt.axis('off')
plt.show()

## 8. Análisis adicional: comparación de detalles

Para apreciar mejor el efecto del filtro pasabajas, podemos observar la diferencia entre la imagen original y la filtrada.

In [ ]:
# Calcular la diferencia entre la imagen original y la filtrada
difference = cv2.absdiff(im_1, im_filtered)

plt.figure(figsize=(4,4))
plt.imshow(difference, cmap='gray')
plt.title('Diferencia entre original y filtrada')
plt.axis('off')
plt.show()